# Modeling the Tower Physics

## Dynamics, Statics, and Friction

Rigid-body dynamics and statics of the tower block friction and deviations from nominal dimensions

Because this modeling framework is more complicated,
analytical solutions may either be intractable or suboptimal to numerical techniques. The Drake physics engine
will be leveraged for this task and to simulate disturbances to the tower.

## Tower Parameters

In [2]:
import numpy as np

# block mass
m = 0.01    # kg
# block length, width, and height
l = 0.1     # m
w = 0.01     # m
h = 0.01     # m
K0 = int(18) # initial tower height
J = int(3) # tower width
n = J*K0 # total number of blocks

# generate block "serial numbers"
block_id = np.random.randint(low=10000,high=99999,size=(n,))

## Initialize the data structure for the tower state.

In [3]:
def initialize_tower(l,w,h,K0,J):
    # initial tower state X
    tower_state = [[{} for _ in range(J)] for _ in range(K0)]

    # Use a for loop to get the initial state of the tower
    count=0
    for k in range(K0):
        for j in range(J):
            if k%2 == 1:
                x_cm = 0.5*l + j*l
                y_cm = 0.5*w + j*w
                #
                x1 = j*l
                x2 = (j+1)*l
                #
                y1 = j*w
                y2 = (j+1)*w
                #
                #z1 = k*h
                #z2 = (k+1)*h
            else:
                x_cm = 0.5*w + j*w
                y_cm = 0.5*l + j*l
                #
                x1 = j*w
                x2 = (j+1)*w
                #
                y1 = j*l
                y2 = (j+1)*l
            edges = [(x1,y1),(x2,y1),(x1,y2),(x2,y2)]  
            z_cm = 0.5*h + k*h
            tower_state[k][j] = {'block': block_id[count],
                                 'XCM': x_cm,'YCM': y_cm, 'ZCM': z_cm,
                                 'edges': edges,
                                 'feasible': False} # may be incorrect on the first iteration
            count +=1
    return tower_state


tower_state = initialize_tower(l,w,h,K0,J)

## Compute the three resultant forces above any given layer.

## Check whether the tower is in static equilibrium.

Firstly, we have a force balance in the vertical direction:

$$
    {}^{k}F_{1}^{i} + {}^{k}F_{2}^{i} + {}^{k}F_{3}^{i} + Mg = {}^{k}f_{1}^{i} + {}^{k}f_{2}^{i} + {}^{k}f_{3}^{i}
$$

For a moment balance, we have:

$$
    {}^{k}F_{1}^{i} - {}^{k}F_{3}^{i} - {}^{k}f_{1}^{i} + {}^{k}f_{3}^{i} = 0
$$

Note that for the moment balance, the level arm distances cancel out, such that we only need to consider the force terms.

Next, we must check that the blocks below the layer $k$ are indeed supporting the weight of the layer $k$. This means that the forces must be greater than zero.

$$
    {}^{k}f_{1}^{i} > 0, {}^{k}f_{2}^{i} > 0, {}^{k}f_{3}^{i} > 0
$$

In [4]:
def check_forces(F1, F2, F3, f1, f2, f3, M):
    g = 9.81
    return (
       f1 >=0 and f2 >=0 and f3 >=0 and
        F1 + F2 + F3 + M*g == f1 + f2 + f3 and
        F1 - F3 - f1 + f3 == 0
    )

